<a href="https://colab.research.google.com/github/freshpex/NLP-Assignment/blob/main/NLP_Assignment_Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data


In [1]:
# import these modules
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))

# a denotes adjective in "pos"
print("better :", lemmatizer.lemmatize("corpora rocks cats", pos="a"))


[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


rocks : rock
corpora : corpus
better : corpora rocks cats


In [2]:
# This dataset contains ~25k tweets in English.
import requests

url = 'https://raw.githubusercontent.com/mhassanist/nlp_text_datasets/ea689be7f27906d0e27b112bc11311d40967b5a3/mini_twitter.txt'
response = requests.get(url)

In [3]:
with open('mini_twitter.txt', 'wb') as file:
    file.write(response.content)

In [4]:
# Open the file and loads its content
text = ''
with open('mini_twitter.txt', 'r', encoding='utf-8') as content_file:
    text = content_file.read()

In [5]:
# Make sure your data is loaded correctly.
# Task 1: Print the first 100 character of the text

# YOUR CODE HERE
print(text[:100])


How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way 


# Import required libraries

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.activations import softmax

# Tokenization

In [7]:
# Task 2.1: Convert the data to lowercase and split it into separate sentences. Store the result in the corpus variable
corpus = text.lower().split("\n")

print(corpus[0]) # how are you? btw thanks for the rt. you gonna be in dc anytime soon? love to see you. been way, way too long.


how are you? btw thanks for the rt. you gonna be in dc anytime soon? love to see you. been way, way too long.


In [8]:
# Task 2.2: Convert all sentences from the text corpus into n-gram sequences of tokens (numeric representations of words)

tokenizer = Tokenizer()

# Fit it on the text
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1


In [9]:
# Task 2.3 : # Use the tokenizer to convert each sentence to n-gram sequence
input_sequences = []

# YOUR CODE HERE
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[1]) # [54, 24, 5]


[56, 22, 5]


# LSTM Model

In [10]:
# Task 3: Create LSTM model, compile it and fit it.
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

x, labels = input_sequences[:,:-1], input_sequences[:,-1]

y = labels


In [11]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

adam = Adam(learning_rate=0.01)


c:\Users\epeki\scoop\apps\python\current\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [12]:
# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x, y, epochs=100, steps_per_epoch=100, verbose=1)


Epoch 1/100
  8/100 ━━━━━━━━━━━━━━━━━━━━ 13:16 9s/step - accuracy: 0.0120 - loss: 10.1908

# Model Prediction

In [ ]:
# Complete the following prediction function

# Prediction function
def predict_next_word(text):
    predicted_word = ''


    # YOUR CODE HERE

    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    predicted_word = tokenizer.index_word[predicted_word_index]


    return predicted_word


# test the model
print(predict_next_word("most people mess up")) # many

print(predict_next_word("most people mess up many")) # many times


# Play with it !

In [ ]:
text = "let's" # starting text
for _ in range(10): # generate the next 10 words
  next = predict_next_word(text)
  text = text + " " + next

print(text)
